In [1]:
import numpy as np
import pandas as pd
import optuna
import dagshub
import mlflow
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from feature_engine.encoding import CountFrequencyEncoder
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_percentage_error

In [2]:
train = pd.read_csv('../data/processed/train.csv', parse_dates=['tpep_pickup_datetime']).drop(columns='month')
test = pd.read_csv('../data/processed/test.csv', parse_dates=['tpep_pickup_datetime']).drop(columns='month')

In [3]:
# adding few more features

# current hour
train['hour_of_day'] = train['tpep_pickup_datetime'].dt.hour
test['hour_of_day'] = test['tpep_pickup_datetime'].dt.hour

# is the day weekend or not, it is important predictor as we saw in eda
train['is_weekend'] = train['day_of_week'].isin([5,6]).values.astype(int)
test['is_weekend'] = test['day_of_week'].isin([5,6]).values.astype(int)

# no. of pickups yesterday
train['pickups_same_time_yesterday'] = train.groupby(['region'])['total_pickups'].shift(96) # 4*24
test['pickups_same_time_yesterday'] = test.groupby(['region'])['total_pickups'].shift(96)

# std of pickup values of last 4 days
train['last_4_days_std'] = train.groupby(['region'])['total_pickups'].rolling(window=4).std().values
test['last_4_days_std'] = test.groupby([ 'region'])['total_pickups'].rolling(window=4).std().values

# removing the null values row, introduced because of shifting of data
train.dropna(inplace=True)
test.dropna(inplace=True)

In [4]:
# shifting the last_4_days_std by 1, if not it will cause data leakage, because we are calculating the std by current value and prev 3 value
# of total_pickups, and current value of total pickups, will not be available during the prediction.

train['last_4_days_std'] = train.groupby('region')['last_4_days_std'].shift(1)

test['last_4_days_std'] = test.groupby('region')['last_4_days_std'].shift(1)

In [5]:
# dropping the null values

train.dropna(inplace=True)
test.dropna(inplace=True)

In [6]:
train.set_index('tpep_pickup_datetime', inplace=True)
test.set_index('tpep_pickup_datetime', inplace=True)

In [7]:
xtrain = train.drop(columns='total_pickups')
xtest = test.drop(columns='total_pickups')

ytrain = train['total_pickups'].copy()
ytest = test['total_pickups'].copy()

In [8]:
cols_to_scale = ['avg_pickups', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'pickups_same_time_yesterday', 'last_4_days_std']

transform = ColumnTransformer(
    transformers=[
        ('scaler', StandardScaler(), cols_to_scale)
    ],
    remainder='passthrough'
)
transform.set_output(transform='pandas')

xtrain_scaled = transform.fit_transform(xtrain)
xtest_scaled = transform.transform(xtest)

col_names = [col.split('__', maxsplit=1)[1] for col in xtrain_scaled.columns]
xtrain_scaled.columns = col_names
xtest_scaled.columns = col_names

In [9]:
cat_cols = ['region', 'day_of_week', 'hour_of_day', 'is_weekend']

xtrain_scaled[cat_cols] = xtrain_scaled[cat_cols].astype('category')
xtest_scaled[cat_cols] = xtest_scaled[cat_cols].astype('category')

In [10]:
xtrain_scaled.dtypes

avg_pickups                     float64
lag_1                           float64
lag_2                           float64
lag_3                           float64
lag_4                           float64
pickups_same_time_yesterday     float64
last_4_days_std                 float64
region                         category
day_of_week                    category
hour_of_day                    category
is_weekend                     category
dtype: object

In [11]:
def SMAPE(y_true, y_pred):
    """
    custom SMAPE loss (Symmetrical Mean Absolute Percentage Error)
    """
    denominator = (np.abs(y_true) + np.abs(y_pred))/2
    diff = np.abs(y_true - y_pred) / denominator
    return np.mean(diff)

In [27]:
def objective_SMAPE(trial):
    """
    Optimizing the algo with SMAPE as loss function
    """
    encoder_type = trial.suggest_categorical("encoder_type", ['Freq', 'Count', 'OHE'])

    if encoder_type == 'OHE':
        encoder = ColumnTransformer(
            transformers=[
                (
                    'OHE', OneHotEncoder(drop='first', sparse_output=False),
                    cat_cols
                )
            ],
            remainder='passthrough'
        )
    
    elif encoder_type == 'Count' : 
        encoder = ColumnTransformer(
            transformers=[
                (
                    'Count Encoder', CountFrequencyEncoder(encoding_method='count'),
                    cat_cols
                )
            ],
            remainder='passthrough'
        )
    
    else:
        encoder = ColumnTransformer(
            transformers=[
                (
                    'Freq Encoder', CountFrequencyEncoder(encoding_method='frequency'),
                    cat_cols
                )
            ],
            remainder='passthrough'
        )
    
    model_type = trial.suggest_categorical("model", ['RF', 'DT', 'XGB'])
    
    if model_type == 'RF':
        model = RandomForestRegressor(
            n_estimators=trial.suggest_int("rf_n_estimators", 50,150),
            max_depth=trial.suggest_categorical("rf_max_depth", [None,5, 30]),
            random_state=42
        )

    elif model_type == 'DT':
        model = DecisionTreeRegressor(
            max_depth=trial.suggest_categorical("rf_max_depth", [None,5, 30]),
            random_state=42
            )
    
    else:
        model = XGBRegressor(
            n_estimators=trial.suggest_int("xgb_n_estimators", 100, 500),
            learning_rate=trial.suggest_float("xgb_learning_rate", 0.01, 0.2),
            max_depth=trial.suggest_categorical("xgb_max_depth", [None, 3, 10]),
            random_state=42
        )
    
    pipe = Pipeline(
        [('encoder', encoder),
        ('model', model)]
    )

    pipe.fit(xtrain_scaled, ytrain)

    ypred = pipe.predict(xtest_scaled)

    return SMAPE(ytest, ypred)

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective_SMAPE, n_trials=25)

In [12]:
mlflow.set_tracking_uri('https://dagshub.com/akshatsharma2407/Urban-Rides-Demand-Prediction-Engine-via-Geo-Clustering.mlflow')
dagshub.init(repo_owner='akshatsharma2407', repo_name='Urban-Rides-Demand-Prediction-Engine-via-Geo-Clustering', mlflow=True)

Accessing as akshatsharma2407

Initialized MLflow to track repo "akshatsharma2407/Urban-Rides-Demand-Prediction-Engine-via-Geo-Clustering"

Repository akshatsharma2407/Urban-Rides-Demand-Prediction-Engine-via-Geo-Clustering initialized!

In [34]:
mlflow.set_experiment(experiment_name='Optuna Experiment 3')
for trial in study.trials:
    with mlflow.start_run(run_name=f'trial_{trial.number}'):
        mlflow.log_params(trial.params)

        if trial.value:
            mlflow.log_metric('objective', trial.value)
        
        mlflow.set_tag('state', trial.state.name)
        mlflow.set_tag('trial_number', trial.number)

2026/01/15 23:58:10 INFO mlflow.tracking.fluent: Experiment with name 'Optuna Experiment 3' does not exist. Creating a new experiment.


🏃 View run trial_0 at: https://dagshub.com/akshatsharma2407/Urban-Rides-Demand-Prediction-Engine-via-Geo-Clustering.mlflow/#/experiments/2/runs/1736ea55748540ce834a229acc58ed14
🧪 View experiment at: https://dagshub.com/akshatsharma2407/Urban-Rides-Demand-Prediction-Engine-via-Geo-Clustering.mlflow/#/experiments/2
🏃 View run trial_1 at: https://dagshub.com/akshatsharma2407/Urban-Rides-Demand-Prediction-Engine-via-Geo-Clustering.mlflow/#/experiments/2/runs/56ec0b3e127148289ba705a6a4ada5fe
🧪 View experiment at: https://dagshub.com/akshatsharma2407/Urban-Rides-Demand-Prediction-Engine-via-Geo-Clustering.mlflow/#/experiments/2
🏃 View run trial_2 at: https://dagshub.com/akshatsharma2407/Urban-Rides-Demand-Prediction-Engine-via-Geo-Clustering.mlflow/#/experiments/2/runs/a11d3ed4b7cb463097b4450746a50ffe
🧪 View experiment at: https://dagshub.com/akshatsharma2407/Urban-Rides-Demand-Prediction-Engine-via-Geo-Clustering.mlflow/#/experiments/2
🏃 View run trial_3 at: https://dagshub.com/akshatsharma

In [13]:
def objective_SMAPE_y_log(trial):
    """
    Optimizing the algo with SMAPE as loss function
    """
    encoder_type = trial.suggest_categorical("encoder_type", ['Freq', 'Count', 'OHE'])

    if encoder_type == 'OHE':
        encoder = ColumnTransformer(
            transformers=[
                (
                    'OHE', OneHotEncoder(drop='first', sparse_output=False),
                    cat_cols
                )
            ],
            remainder='passthrough'
        )
    
    elif encoder_type == 'Count' : 
        encoder = ColumnTransformer(
            transformers=[
                (
                    'Count Encoder', CountFrequencyEncoder(encoding_method='count'),
                    cat_cols
                )
            ],
            remainder='passthrough'
        )
    
    else:
        encoder = ColumnTransformer(
            transformers=[
                (
                    'Freq Encoder', CountFrequencyEncoder(encoding_method='frequency'),
                    cat_cols
                )
            ],
            remainder='passthrough'
        )
    
    model_type = trial.suggest_categorical("model", ['RF', 'DT', 'XGB'])
    
    if model_type == 'RF':
        model = RandomForestRegressor(
            n_estimators=trial.suggest_int("rf_n_estimators", 50,150),
            max_depth=trial.suggest_categorical("rf_max_depth", [None,5, 30]),
            random_state=42
        )

    elif model_type == 'DT':
        model = DecisionTreeRegressor(
            max_depth=trial.suggest_categorical("rf_max_depth", [None,5, 30]),
            random_state=42
            )
    
    else:
        model = XGBRegressor(
            n_estimators=trial.suggest_int("xgb_n_estimators", 100, 500),
            learning_rate=trial.suggest_float("xgb_learning_rate", 0.01, 0.2),
            max_depth=trial.suggest_categorical("xgb_max_depth", [None, 3, 10]),
            random_state=42
        )
    
    pipe = Pipeline(
        [('encoder', encoder),
        ('model', model)]
    )

    ytrain_log = np.log1p(ytrain)

    pipe.fit(xtrain_scaled, ytrain_log)

    ypred_log = pipe.predict(xtest_scaled)

    ypred = np.expm1(ypred_log)

    return SMAPE(ytest, ypred)

In [14]:
study = optuna.create_study(direction='minimize')
study.optimize(objective_SMAPE_y_log, n_trials=50, n_jobs=-1)

[I 2026-01-16 00:24:56,899] A new study created in memory with name: no-name-0e500b4d-6895-417a-aee2-561beb3a0d21
[I 2026-01-16 00:24:59,276] Trial 5 finished with value: 0.22926247090480226 and parameters: {'encoder_type': 'Count', 'model': 'DT', 'rf_max_depth': 5}. Best is trial 5 with value: 0.22926247090480226.
[I 2026-01-16 00:25:00,506] Trial 6 finished with value: 0.22926247090480265 and parameters: {'encoder_type': 'OHE', 'model': 'DT', 'rf_max_depth': 5}. Best is trial 5 with value: 0.22926247090480226.
[I 2026-01-16 00:25:01,152] Trial 2 finished with value: 0.28296682536120965 and parameters: {'encoder_type': 'Freq', 'model': 'DT', 'rf_max_depth': 30}. Best is trial 5 with value: 0.22926247090480226.
[I 2026-01-16 00:25:02,118] Trial 3 finished with value: 0.28402204065258135 and parameters: {'encoder_type': 'Freq', 'model': 'DT', 'rf_max_depth': None}. Best is trial 5 with value: 0.22926247090480226.
[I 2026-01-16 00:25:02,792] Trial 0 finished with value: 0.282966825361209

In [15]:
mlflow.set_experiment(experiment_name='Optuna Experiment 4')
for trial in study.trials:
    with mlflow.start_run(run_name=f'trial_{trial.number}'):
        mlflow.log_params(trial.params)

        if trial.value:
            mlflow.log_metric('objective', trial.value)
        
        mlflow.set_tag('state', trial.state.name)
        mlflow.set_tag('trial_number', trial.number)

2026/01/16 00:41:51 INFO mlflow.tracking.fluent: Experiment with name 'Optuna Experiment 4' does not exist. Creating a new experiment.


🏃 View run trial_0 at: https://dagshub.com/akshatsharma2407/Urban-Rides-Demand-Prediction-Engine-via-Geo-Clustering.mlflow/#/experiments/3/runs/ffeb8d3566c54caa8d830f122077c15d
🧪 View experiment at: https://dagshub.com/akshatsharma2407/Urban-Rides-Demand-Prediction-Engine-via-Geo-Clustering.mlflow/#/experiments/3
🏃 View run trial_1 at: https://dagshub.com/akshatsharma2407/Urban-Rides-Demand-Prediction-Engine-via-Geo-Clustering.mlflow/#/experiments/3/runs/3a0af31d88654996a45dedab37c88f50
🧪 View experiment at: https://dagshub.com/akshatsharma2407/Urban-Rides-Demand-Prediction-Engine-via-Geo-Clustering.mlflow/#/experiments/3
🏃 View run trial_2 at: https://dagshub.com/akshatsharma2407/Urban-Rides-Demand-Prediction-Engine-via-Geo-Clustering.mlflow/#/experiments/3/runs/cfcdb3a932b548ab8e8eb89c1c898ad3
🧪 View experiment at: https://dagshub.com/akshatsharma2407/Urban-Rides-Demand-Prediction-Engine-via-Geo-Clustering.mlflow/#/experiments/3
🏃 View run trial_3 at: https://dagshub.com/akshatsharma

In [17]:
train.to_csv('../data/processed/train_enhanced_features.csv', index=True)
test.to_csv('../data/processed/test_enhanced_features.csv', index=True)